In [ ]:
import sklearn as sk
import os

In [ ]:
path_neg_true = './database/spam/negative_polarity/truthful_from_Web'
path_neg_false = './database/spam/negative_polarity/deceptive_from_MTurk'

path_pos_true = './database/spam/positive_polarity/truthful_from_TripAdvisor'
path_pos_false = './database/spam/positive_polarity/deceptive_from_MTurk'
  
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk import ngrams
import string

def read_range(data,a,b,n, label):
    res = []
    for i in range(a,b):
        folder = 'fold'+ str(i)
        for file in os.listdir(data + "/" + folder):
             with open(data + '/' + folder + '/' + file, 'r') as content_file:
                content = content_file.read()
                res.append(content)
    return res

def read_dir(data,n, label):
  return read_range(data,1,5,n,label),read_range(data,5,6,n,label)

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from nltk.classify import NaiveBayesClassifier
from sklearn.svm import SVC

scores = ['accuracy']
n=1
train_neg_true,test_neg_true = read_dir(path_neg_true,n, 'positive')
train_neg_false,test_neg_false = read_dir(path_neg_false,n,'negative')

dfTrue = pd.DataFrame(train_neg_true)
dfTrue['y'] = 1

dfFalse = pd.DataFrame(train_neg_false)
dfFalse['y'] = 0


dfTestTrue = pd.DataFrame(test_neg_true)
dfTestTrue['y']=1

dfTestFalse = pd.DataFrame(test_neg_false)
dfTestFalse['y'] = 0

frames = [dfTrue,dfFalse]
test_frames = [dfTestTrue,dfTestFalse]

df = pd.concat(frames)
test_df = pd.concat(test_frames)

#Naive Bayes classifier


In [ ]:
text_clfMNB = Pipeline([ ('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
])

text_clfLR = Pipeline([ ('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', LogisticRegression(random_state=42))
])

text_clfCT = Pipeline([ ('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', tree.DecisionTreeClassifier())
])

text_clfRF = Pipeline([ ('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', ExtraTreesClassifier(random_state=42))
])

In [ ]:
def run_pipeline(text_clf,parameters):
    pipe = text_clf.fit(df[0],df['y'])  
    predicted = pipe.predict(test_df[0])
    gs_clf = GridSearchCV(text_clf, parameters, cv=5)
    gs_clf = gs_clf.fit(df[0],df['y'])
    predicted = gs_clf.predict(test_df[0])
    print(metrics.classification_report(test_df['y'], predicted))
    #print(gs_clf.best_score_)
    #print(gs_clf.best_estimator_)
    
    return gs_clf.best_estimator_

In [ ]:
parameters_MNB = {
               'vect__ngram_range': [(1, 1), (2, 2)],
               'tfidf__use_idf': (True, False),
               'clf__alpha': (1e-2, 1e-3),
}

parameters_DT = {
               'vect__ngram_range': [(1, 1), (2, 2)],
               'tfidf__use_idf': (True, False),
}

parameters_LR = {
               'vect__ngram_range': [(1, 1), (2, 2)],
               'tfidf__use_idf': (True, False),
                'clf__C': [1,0.1,0.001,0.0001],
}

parameters_RF = {
               'vect__ngram_range': [(1, 1), (2, 2)],
               'tfidf__use_idf': (True, False),
               'clf__n_estimators' : [100, 200, 300],
               'clf__max_depth' : [2,3,4,5]
}

In [38]:
print('-------Naive Bayes--------')
mnb = run_pipeline(text_clfMNB,parameters_MNB)

-------Naive Bayes--------
             precision    recall  f1-score   support

          0       0.90      0.88      0.89        80
          1       0.88      0.90      0.89        80

avg / total       0.89      0.89      0.89       160



In [ ]:
print('--------Dec. Trees------')
best_tree = run_pipeline(text_clfCT,parameters_DT)

In [67]:
print('-------Random forests------')
import matplotlib.pyplot as plt
import numpy as np
#best_forest = run_pipeline(text_clfRF,parameters_RF)

best_forest = run_pipeline(text_clfRF,{})
forest = best_forest.steps[len(best_forest.steps)-1][1]

importante = forest.feature_importances_
importances = [x for x in importante if x > 0.0009]
indices = np.argsort(importances)[::-1]
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
print(indices)
print(importances)


-------Random forests------
             precision    recall  f1-score   support

          0       0.70      0.75      0.72        80
          1       0.73      0.68      0.70        80

avg / total       0.71      0.71      0.71       160

[ 55 298  29 158 220 234 153 304 219  73 226  90 107 173 210  20 101 270
  18  25 124 200 115 273 314  52 241  93 277  59 169 159 309 188 126   7
 172  37 303  89 207   3  16  68 235 192  92 162 269 199 125 177  94  53
 105  28 225 120 217  84 271   9 266  61  60 157 223  48 221  13 198  22
  91 211 131 154  47 259 242  79  33  23 248 260 305 295   8 267  14  19
 247 261 174  17 194 134 249 204 236 136  38  71 163 312 112 265 127  76
  57 246 296 254 168 108 213 250  30 190 287 166 251  10 147 193 231 212
 208  11 318 268 245  63 187 149 209  62 161 132 313  97 145  45 275 144
   2  34 257 222 281 116 278  95 106 306 272 129  56  50  27 150 262   4
 151 215  78 297 244 233 299  83 276 317 185 284 176  70  74 291 135 156
  67  58 205  49   5  65  4

In [ ]:
print('--------Logistic regression-------')
run_pipeline(text_clfLR, parameters_LR)